# Mixer-TTS Training

This notebook is designed to provide a guide on how to train Mixer-TTS as part of the TTS pipeline. It contains the following two sections:
  1. **Introduction**: Mixer-TTS in NeMo
  2. **Preprocessing**: how to prepare data for Mixer-TTS 
  3. **Training**: example of Mixer-TTS training

# License

> Copyright 2021 NVIDIA. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies# .
"""
# # If you're using Colab and not running locally, uncomment and run this cell.
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode
# BRANCH = 'main'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
import json
import nemo
import torch
import librosa
import numpy as np

from pathlib import Path
from tqdm.notebook import tqdm

# Introduction

Mixer-TTS is non-autoregressive model for mel-spectrogram generation. The model is based on the MLP-Mixer architecture adapted for speech synthesis. It contains pitch and duration predictors, with the latter being trained with an unsupervised TTS alignment framework.
For more details about model, please refer to the original [paper](https://arxiv.org/abs/2110.03584).

Mixer-TTS like most NeMo models is defined as a LightningModule, allowing for easy training via PyTorch Lightning, and parameterized by a configuration, currently defined via a yaml file and loading using Hydra.

Let's take a look using NeMo's pretrained model and how to use it to generate spectrograms.

In [ ]:
# Load the MixerTTSModel
from nemo.collections.tts.models import MixerTTSModel
from nemo.collections.tts.models.base import SpectrogramGenerator

# Let's see what pretrained models are available
print(MixerTTSModel.list_available_models())

In [ ]:
# We can load the pre-trained model as follows
pretrained_model = "tts_en_lj_mixertts"
model = MixerTTSModel.from_pretrained(pretrained_model)
model.eval();

In [ ]:
# MixerTTSModel is a SpectrogramGenerator
assert isinstance(model, SpectrogramGenerator)

# SpectrogramGenerators in NeMo have two helper functions:
#   1. parse(text: str, **kwargs) which takes string and produces a token tensor
#   2. generate_spectrogram(tokens: 'torch.tensor', **kwargs) which takes the token tensor and generates a spectrogram
# Let's try it out
tokens = model.parse(text="Hey, this produces speech!")
spectrogram = model.generate_spectrogram(tokens=tokens)

# Now we can visualize the generated spectrogram
# If we want to generate speech, we have to use a vocoder in conjunction to a spectrogram generator.
# Refer to the Inference_ModelSelect notebook on how to convert spectrograms to speech.
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
%matplotlib inline
imshow(spectrogram.cpu().detach().numpy()[0,...], origin="lower")
plt.show()

# Preprocessing

Now that we looked at the Mixer-TTS model, let's see how to prepare all data for training it. 

Firstly, let's download all necessary training scripts and configs.

In [ ]:
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/mixer_tts.py

!mkdir -p conf && cd conf \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/tts/conf/mixer-tts.yaml \
&& cd ..

We will show example of preprocessing and training using small part of AN4 dataset. It consists of recordings of people spelling out addresses, names, telephone numbers, etc., one letter or number at a time, as well as their corresponding transcripts. Let's download data, prepared manifests and supplementary files.

*NOTE: The sample data is not enough data to properly train a Mixer-TTS. This will not result in a trained Mixer-TTS and is used to just as example.*

In [ ]:
# download data and manifests
!wget https://github.com/NVIDIA/NeMo/releases/download/v0.11.0/test_data.tar.gz && mkdir -p tests/data && tar xzf test_data.tar.gz -C tests/data

# Mixer-TTS can be trained on phonemes and uses normalization tool, let's download additional data for them
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/nemo_text_processing/text_normalization/en/data/whitelist_lj_speech.tsv \
&& cd ..

TTS text preprocessing pipeline consists of two stages: text normalization and text tokenization. Both of them can be handled by `nemo.collections.tts.torch.data.TTSDataset` for training.  

Our current example dataset is in English, so let's use `nemo_text_processing.text_normalization.normalize.Normalizer` for normalization which supports English (and many other languages!) and `nemo.collections.tts.torch.tts_tokenizers.EnglishPhonemesTokenizer` for tokenization which can work together with grapheme-to-phoneme (G2P) module. For G2P we use `nemo.collections.tts.torch.g2ps.EnglishG2p`.

In [ ]:
from nemo.collections.tts.torch.g2ps import EnglishG2p
from nemo.collections.tts.torch.data import TTSDataset
from nemo_text_processing.text_normalization.normalize import Normalizer
from nemo.collections.tts.torch.tts_tokenizers import EnglishPhonemesTokenizer

In [ ]:
# text normalizer
text_normalizer = Normalizer(
    lang="en", 
    input_case="cased", 
    whitelist="tts_dataset_files/whitelist_lj_speech.tsv"
)

text_normalizer_call_kwargs = {
    "punct_pre_process": True,
    "punct_post_process": True
}

# grapheme-to-phoneme module
g2p = EnglishG2p(
    phoneme_dict="tts_dataset_files/cmudict-0.7b",
    heteronyms="tts_dataset_files/heteronyms-030921"
)

# text tokenizer
text_tokenizer = EnglishPhonemesTokenizer(
    punct=True,
    stresses=True,
    chars=True,
    apostrophe=True,
    pad_with_space=True,
    g2p=g2p,
)

To accelerate and stabilize our training, we also need to extract pitch for every audio, estimate pitch statistics (mean and std) and pre-calculate alignment prior matrices. To do this, all we need to do is iterate over our data one time.

In [ ]:
sup_data_path = "sup_data_folder"
sup_data_types = ["align_prior_matrix", "pitch"]

# init train and val dataloaders
stages = ["train", "val"]
stage2dl = {}
for stage in stages:
    ds = TTSDataset(
        manifest_filepath=f"tests/data/asr/an4_{stage}.json",
        sample_rate=16000,
        sup_data_path=sup_data_path,
        sup_data_types=sup_data_types,
        n_fft=1024,
        win_length=1024,
        hop_length=256,
        window="hann",
        n_mels=80,
        lowfreq=0,
        highfreq=8000,
        text_tokenizer=text_tokenizer,
        text_normalizer=text_normalizer,
        text_normalizer_call_kwargs=text_normalizer_call_kwargs

    ) 
    stage2dl[stage] = torch.utils.data.DataLoader(ds, batch_size=1, collate_fn=ds._collate_fn, num_workers=1)

# iteration over dataloaders
for stage, dl in stage2dl.items():
    pitch_list = []
    for batch in tqdm(dl, total=len(dl)):
        tokens, tokens_lengths, audios, audio_lengths, attn_prior, pitches, pitches_lengths = batch
        pitch = pitches.squeeze(0)
        pitch_list.append(pitch[pitch != 0])
    
    if stage == "train":
        pitch_tensor = torch.cat(pitch_list)
        pitch_mean, pitch_std = pitch_tensor.mean().item(), pitch_tensor.std().item()

# Training

Now we are ready for training our model! Let's try to train Mixer-TTS.

*NOTE: The sample data is not enough data to properly train a Mixer-TTS. This will not result in a trained Mixer-TTS and is used to just as example.*

In [ ]:
!python mixer_tts.py sample_rate=16000 \
train_dataset=tests/data/asr/an4_train.json \
validation_datasets=tests/data/asr/an4_val.json \
sup_data_types="['align_prior_matrix', 'pitch']" \
sup_data_path={sup_data_path} \
phoneme_dict_path=tts_dataset_files/cmudict-0.7b \
heteronyms_path=tts_dataset_files/heteronyms-030921 \
whitelist_path=tts_dataset_files/whitelist_lj_speech.tsv \
pitch_mean={pitch_mean} \
pitch_std={pitch_std} \
model.train_ds.dataloader_params.batch_size=6 \
model.train_ds.dataloader_params.num_workers=0 \
model.validation_ds.dataloader_params.num_workers=0 \
trainer.max_epochs=3 \
trainer.strategy=null \
trainer.check_val_every_n_epoch=1

That's it!

In order to train Mixer-TTS for real purposes, it is highly recommended to obtain high quality speech data with the following properties:

* Sampling rate of 22050Hz or higher
* Single speaker
* Speech should contain a variety of speech phonemes
* Audio split into segments of 1-10 seconds
* Audio segments should not have silence at the beginning and end
* Audio segments should not contain long silences inside